# Boltzmann Machine

## Importing the libraries

In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset

In [9]:
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [11]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [13]:
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [15]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [17]:
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [19]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## Preparing the training set and test set

In [22]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')

In [27]:
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [24]:
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [31]:
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

## Getting the number of users and movies

In [33]:
num_users = len(np.unique(np.concatenate((training_set[:, 0], test_set[:, 0]), axis = 0)))
num_movies = len(np.unique(np.concatenate((training_set[:, 1], test_set[:, 1]), axis = 0)))

In [35]:
num_users, num_movies

(943, 1682)

## Converting the data into an array with users in rows and movies in columns

In [38]:
def convert(data):
    new_data = []
    for id_users in range(1, num_users+1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(num_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data        

In [40]:
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors

In [47]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Converting the ratings into binary ratings 1(Liked) or 0(Not liked)

In [50]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

## Creating the architecture of the RBM

In [66]:
class RBM():
    def __init__(self, nv, nh):
         self.W = torch.randn(nh, nv)
         self.a = torch.randn(1, nh)
         self.b = torch.randn(1, nv)
    def sample_h(self, x):
         wx = torch.mm(x, self.W.t())
         activation = wx + self.a.expand_as(wx)
         p_h_given_v = torch.sigmoid(activation)
         return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
         self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
         self.b += torch.sum((v0 - vk), 0)
         self.a += torch.sum((ph0 - phk), 0)

In [68]:
nv = num_movies
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

## Training the RBM

In [70]:
num_epochs = 10
for epoch in range(1, num_epochs+1):
    train_loss = 0
    s = 0.
    for id_user in range(0, num_users-batch_size, 100):
        vk = training_set[id_user: id_user+batch_size]
        v0 = training_set[id_user: id_user+batch_size]
        ph0, _ = rbm.sample_h(v0)
        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)
            vk[v0 < 0] = v0[v0 < 0]
        phk, _ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        s+=1
    print(f'epoch : {epoch} loss: {train_loss/s}')    

epoch : 1 loss: 0.3506610095500946
epoch : 2 loss: 0.249216690659523
epoch : 3 loss: 0.24559849500656128
epoch : 4 loss: 0.2509387731552124
epoch : 5 loss: 0.24805767834186554
epoch : 6 loss: 0.2516283690929413
epoch : 7 loss: 0.2522052526473999
epoch : 8 loss: 0.2484803944826126
epoch : 9 loss: 0.24812409281730652
epoch : 10 loss: 0.24435168504714966


## Testing the RBM

In [73]:
test_loss = 0
s = 0.
for id_user in range(num_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2444)
